In [1]:
import requests
import pandas as pd
from io import BytesIO

In [37]:
def descargar_indices_precios(series, anio_inicial, anio_final):
    """
    series: lista de IDs o string
    ejemplo: ["1730466", "1730470"]
    """

    # convertir lista a string
    if isinstance(series, list):
        series_str = ",".join(series) + ","
    else:
        series_str = series + ","

    url = (
        "https://www.inegi.org.mx/app/indicesdepreciosv2/Exportacion.aspx"
        f"?_series=e|{series_str}&_formato=XLS&_orient=vertical"
        f"&_anioI={anio_inicial}&_anioF={anio_final}"
        "&_meta=1&_tipo=Niveles&_info=Índices"
    )

    headers = {"User-Agent": "Mozilla/5.0"}
    r = requests.get(url, headers=headers)

    df_raw = pd.read_excel(BytesIO(r.content))
        # eliminar primeras 15 filas
    df = df_raw.iloc[15:].reset_index(drop=True)
        # renombrar primera columna como fecha
    df = df.rename(columns={df.columns[0]: "fecha"})
        # convertir fecha
    #df["fecha"] = pd.to_datetime(df["fecha"], format="%b %Y", errors="coerce")
    return df

In [96]:
def descargar_indices_precios(series, anio_inicial, anio_final):

    if isinstance(series, str):
        series = [series]

    # mapa de estructuras por serie
    estructura_map = {
        "1730466": "112001900050",
        "1730468": "112001900050"
    }

    # agrupar series por estructura
    grupos = {}
    for s in series:
        estructura = estructura_map.get(s, "112001700030")
        grupos.setdefault(estructura, []).append(s)

    dfs = []

    def descargar(series_list, estructura):
        series_str = ",".join(series_list) + ","

        if estructura == "112001900050":
            extra = (
                "&_meta=1&_tipo=Tipo de información"
                "&_info=Tipo de información"
                "&st=Producción total, según actividad económica de origen SCIAN 2018"
                f"&idEstructura={estructura}"
            )
        else:
            extra = (
                "&_meta=1&_tipo=Tipo de información"
                "&_info=Tipo de información"
                "&st="
                f"&idEstructura={estructura}"
            )

        url = (
            "https://www.inegi.org.mx/app/indicesdepreciosv2/Exportacion.aspx"
            f"?_series=e|{series_str}&_formato=XLS&_orient=vertical"
            f"&_anioI={anio_inicial}&_anioF={anio_final}"
            f"{extra}"
        )

        print(url)

        headers = {"User-Agent": "Mozilla/5.0"}
        r = requests.get(url, headers=headers)

        df_raw = pd.read_excel(BytesIO(r.content))
        df = df_raw.iloc[15:].reset_index(drop=True)
        df = df.rename(columns={df.columns[0]: "fecha"})

        meses = {
            "Ene": "Jan", "Feb": "Feb", "Mar": "Mar", "Abr": "Apr",
            "May": "May", "Jun": "Jun", "Jul": "Jul", "Ago": "Aug",
            "Sep": "Sep", "Oct": "Oct", "Nov": "Nov", "Dic": "Dec"
        }

        df["fecha"] = (
            df["fecha"]
            .replace(meses, regex=True)
            .pipe(pd.to_datetime, format="%b %Y", errors="coerce")
        )

        for col in df.columns:
            if col != "fecha":
                df[col] = pd.to_numeric(df[col], errors="coerce")

        df = df.dropna(subset=["fecha"])
        return df

    # descargar por cada estructura
    for estructura, series_list in grupos.items():
        dfs.append(descargar(series_list, estructura))

    # unir todos los resultados
    df_final = dfs[0]
    for df in dfs[1:]:
        df_final = df_final.merge(df, on="fecha", how="outer")

    return df_final.sort_values("fecha").reset_index(drop=True)


In [97]:
path_iqy = r"C:\Users\claud\downloads\INP_c20260210183233.iqy"
anio_inicial = 1981
anio_final = 2025

# -----------------------------------
# 1. Leer archivo IQY
# -----------------------------------
with open(path_iqy, "r", encoding="latin-1") as f:
    contenido = "".join(f.readlines())

print(contenido)

https://www.inegi.org.mx/app/indicesdepreciosv2/Exportacion.aspx
_series=e|865587,865758,865815,865840,865895,865951,865987,866034,&_formato=IQY&_orient=vertical&_anioI=["añoInicial","Año inicial"]&_anioF=["añoFinal","Año final"]&_meta=1&_tipo=Tipo de información&_info=Tipo de información&st=&idEstructura=112001700030


In [98]:
path_iqy = r"C:\Users\claud\downloads\INP_c20260210185912.iqy"
anio_inicial = 1981
anio_final = 2025

# -----------------------------------
# 1. Leer archivo IQY
# -----------------------------------
with open(path_iqy, "r", encoding="latin-1") as f:
    contenido = "".join(f.readlines())

print(contenido)

https://www.inegi.org.mx/app/indicesdepreciosv2/Exportacion.aspx
_series=e|1730466,1730468,&_formato=IQY&_orient=vertical&_anioI=["añoInicial","Año inicial"]&_anioF=["añoFinal","Año final"]&_meta=1&_tipo=Tipo de información&_info=Tipo de información&st=Producción total, según actividad económica de origen SCIAN 2018&idEstructura=112001900050


In [101]:
indices=descargar_indices_precios(["1730466","1730468","865587","865758","865815","865840","865895","865951","865987","866034"], 1981, 2025)
#indices=descargar_indices_precios(["1730466","1730468"], 1981, 2025)
#Renombrar columnas después de fecha: ferrocarril, autotransporte_carga, alimentos,
nuevos_nombres = [
    "ferrocarril",
    "autotransporte_carga",
    "alimentos_bebidas_tabaco",
    "ropa_calzado_accesorios",
    "vivienda",
    "muebles_aparatos_domesticos",
    "salud_cuidado_personal",
    "transporte",
    "educacion_esparcimiento",
    "otros_servicios"
]

indices.columns = ["fecha"] + nuevos_nombres

indices

https://www.inegi.org.mx/app/indicesdepreciosv2/Exportacion.aspx?_series=e|1730466,1730468,&_formato=XLS&_orient=vertical&_anioI=1981&_anioF=2025&_meta=1&_tipo=Tipo de información&_info=Tipo de información&st=Producción total, según actividad económica de origen SCIAN 2018&idEstructura=112001900050
https://www.inegi.org.mx/app/indicesdepreciosv2/Exportacion.aspx?_series=e|865587,865758,865815,865840,865895,865951,865987,866034,&_formato=XLS&_orient=vertical&_anioI=1981&_anioF=2025&_meta=1&_tipo=Tipo de información&_info=Tipo de información&st=&idEstructura=112001700030


,fecha,ferrocarril,autotransporte_carga,alimentos_bebidas_tabaco,ropa_calzado_accesorios,vivienda,muebles_aparatos_domesticos,salud_cuidado_personal,transporte,educacion_esparcimiento,otros_servicios
0,1981-01-01,0.035212,0.091011,0.061697,0.119081,0.087518,0.119698,0.054835,0.039152,0.049684,0.038335
1,1981-02-01,0.043321,0.091011,0.062759,0.121936,0.089835,0.123111,0.057303,0.041349,0.050384,0.039296
2,1981-03-01,0.043321,0.091011,0.063860,0.124959,0.092290,0.126239,0.058943,0.042185,0.051456,0.040152
3,1981-04-01,0.043321,0.091011,0.065104,0.128513,0.094137,0.129158,0.060004,0.042927,0.052612,0.041652
4,1981-05-01,0.043321,0.091011,0.066176,0.131134,0.095002,0.131304,0.061040,0.043346,0.053568,0.042245
...,...,...,...,...,...,...,...,...,...,...,...
535,2025-08-01,125.134371,135.801176,160.956000,125.260000,121.293000,123.344000,146.522000,132.374000,127.729000,163.387000
536,2025-09-01,125.588963,136.177255,161.115000,125.852000,121.454000,123.515000,147.227000,132.396000,129.480000,163.364000
537,2025-10-01,125.588963,136.287707,160.860000,126.086000,122.778000,123.624000,147.350000,132.924000,129.929000,164.249000
538,2025-11-01,125.588963,136.918636,161.756000,125.519000,124.468000,123.378000,147.599000,133.974000,129.881000,165.882000


In [102]:
indices[indices["fecha"] == "2019-07-01"]

,fecha,ferrocarril,autotransporte_carga,alimentos_bebidas_tabaco,ropa_calzado_accesorios,vivienda,muebles_aparatos_domesticos,salud_cuidado_personal,transporte,educacion_esparcimiento,otros_servicios
462,2019-07-01,100.0,100.0,105.199,102.265,101.256,100.917,104.46,104.33,103.609,105.529


### Cambio de base

In [105]:
# tomar la fila base
base = indices.loc[indices["fecha"] == "2019-07-01"].iloc[0]

# copiar dataframe
indices_base_2019 = indices.copy()

# columnas numéricas (todas menos fecha)
cols = indices.columns.drop("fecha")

# rebasing
indices_base_2019[cols] = (
    indices_base_2019[cols]
    .div(base[cols])
    .mul(100)
)
indices_base_2019

,fecha,ferrocarril,autotransporte_carga,alimentos_bebidas_tabaco,ropa_calzado_accesorios,vivienda,muebles_aparatos_domesticos,salud_cuidado_personal,transporte,educacion_esparcimiento,otros_servicios
0,1981-01-01,0.035212,0.091011,0.058648,0.116444,0.086432,0.11861,0.052494,0.037527,0.047953,0.036326
1,1981-02-01,0.043321,0.091011,0.059657,0.119236,0.088721,0.121993,0.054856,0.039633,0.048629,0.037237
2,1981-03-01,0.043321,0.091011,0.060704,0.122192,0.091145,0.125092,0.056426,0.040434,0.049664,0.038048
3,1981-04-01,0.043321,0.091011,0.061886,0.125666,0.092969,0.127984,0.057442,0.041146,0.050779,0.03947
4,1981-05-01,0.043321,0.091011,0.062905,0.12823,0.093824,0.130111,0.058434,0.041547,0.051702,0.040032
...,...,...,...,...,...,...,...,...,...,...,...
535,2025-08-01,125.134371,135.801176,153.001454,122.485699,119.788457,122.223213,140.266131,126.880092,123.279831,154.826635
536,2025-09-01,125.588963,136.177255,153.152597,123.064587,119.94746,122.392659,140.94103,126.901179,124.969839,154.80484
537,2025-10-01,125.588963,136.287707,152.910199,123.293404,121.255037,122.500669,141.058778,127.407265,125.403199,155.643472
538,2025-11-01,125.588963,136.918636,153.761918,122.738962,122.924074,122.256904,141.297147,128.413687,125.356871,157.190914


In [106]:
indices_base_2019[indices_base_2019["fecha"] == "2019-07-01"]

,fecha,ferrocarril,autotransporte_carga,alimentos_bebidas_tabaco,ropa_calzado_accesorios,vivienda,muebles_aparatos_domesticos,salud_cuidado_personal,transporte,educacion_esparcimiento,otros_servicios
462,2019-07-01,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0


In [107]:
## Salvar archivo de excel
indices_base_2019.to_excel("indices_precios_base_2019.xlsx", index=False)